## Import libraries

In [ ]:
import boto3
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader

sess = boto3.Session()
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = f'afd-{sess.region_name}-{account_id}'
role = get_execution_role()

## Create S3 bucket

In [ ]:
try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=bucket)
    else:
        sess.client('s3').create_bucket(Bucket=bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print("Looks like you already have a bucket of this name. That's good. Uploading the data files...")

## Upload the training data

In [ ]:
s3url = S3Uploader.upload('../1 Data Preparation/data/afd_training_data.csv', f's3://{bucket}/training')
print(s3url)

## Create Fraud Detetcor instance

In [ ]:
fraudDetector = boto3.client('frauddetector')

## Create model variables

In [ ]:
try:
    fraudDetector.create_variable(
    name = 'registration_time',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
    )
except Exception as e:
    print("Looks like you already have this variable. That's good")

try:
    fraudDetector.create_variable(
    name = 'email_address',
    variableType = 'EMAIL_ADDRESS',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
    )
except Exception as e:
    print("Looks like you already have this variable. That's good")
    
try:
    fraudDetector.create_variable(
    name = 'ip_address',
    variableType = 'IP_ADDRESS',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
    )
except Exception as e:
    print("Looks like you already have this variable. That's good")

## Create model outcomes

In [ ]:
try:
    fraudDetector.put_outcome(
    name = 'review',
    description = 'this outcome sidelines event for review'
    )
except Exception as e:
    print("Looks like you already have this outcome. That's good")

try:
    fraudDetector.put_outcome(
    name = 'approve',
    description = 'this outcome approves the event'
    )
except Exception as e:
    print("Looks like you already have this outcome. That's good")

## Create customer entitites

In [ ]:
try:
    fraudDetector.put_entity_type(
    name = 'sample_customer',
    description = 'sample customer entity type'
    )
except Exception as e:
    print("Looks like you already have this entity. That's good")

## Create labels

In [ ]:
try:
    fraudDetector.put_label(
    name = 'fraud',
    description = 'label for fraud events'
    )
except Exception as e:
    print("Looks like you already have this label. That's good")

try:
    fraudDetector.put_label(
    name = 'legit',
    description = 'label for legitimate events'
    )
except Exception as e:
    print("Looks like you already have this label. That's good")

## Create event type

In [ ]:
try:
    fraudDetector.put_event_type (
    name = 'sample_registration',
    eventVariables = ['ip_address', 'email_address', 'registration_time'],
    labels = ['legit', 'fraud'],
    entityTypes = ['sample_customer'])
except Exception as e:
    print("Looks like you already have this event. That's good")

## Create model

In [ ]:
try:
    fraudDetector.create_model (
    modelId = 'sample_fraud_detection_model',
    eventTypeName = 'sample_registration',
    modelType = 'ONLINE_FRAUD_INSIGHTS')
except Exception as e:
    print("Looks like you already have this model. That's good")

## Create model version - This will kick off the model training

In [ ]:
fraudDetector.create_model_version (
modelId = 'sample_fraud_detection_model',
modelType = 'ONLINE_FRAUD_INSIGHTS',
trainingDataSource = 'EXTERNAL_EVENTS',
trainingDataSchema = {
    'modelVariables' : ['ip_address', 'email_address', 'registration_time'],
    'labelSchema' : {
        'labelMapper' : {
            'FRAUD' : ['fraud'],
            'LEGIT' : ['legit']
        }
    }
}, 
externalEventsDetail = {
    'dataLocation' : s3url,
    'dataAccessRoleArn' : role
}
)

## Check training status

In [ ]:
import time
while True:
    r = fraudDetector.get_model_version(
        modelId='sample_fraud_detection_model',
        modelType='ONLINE_FRAUD_INSIGHTS',
        modelVersionNumber='1.0')
    
    print(r['status'])
    
    if r['status'] != 'TRAINING_IN_PROGRESS':
        break
    
    time.sleep(300)

## Activate the new model version

In [ ]:
fraudDetector.update_model_version_status (
modelId = 'sample_fraud_detection_model',
modelType = 'ONLINE_FRAUD_INSIGHTS',
modelVersionNumber = '1.00',
status = 'ACTIVE'
)

## Create a detector

In [ ]:
fraudDetector.put_detector (
detectorId = 'sample_detector',
eventTypeName = 'sample_registration'
)

## Create rules based on model scores

In [ ]:
try:
    fraudDetector.create_rule(
    ruleId = 'high_fraud_risk',
    detectorId = 'sample_detector',
    expression = '$sample_fraud_detection_model_insightscore > 900',
    language = 'DETECTORPL',
    outcomes = ['review']
    )
except Exception as e:
    print("Looks like you already have this rule. That's good")

try:
    fraudDetector.create_rule(
    ruleId = 'low_fraud_risk',
    detectorId = 'sample_detector',
    expression = '$sample_fraud_detection_model_insightscore <= 900',
    language = 'DETECTORPL',
    outcomes = ['approve']
    )
except Exception as e:
    print("Looks like you already have this rule. That's good")

## Create a detector version

In [ ]:
fraudDetector.create_detector_version(
detectorId = 'sample_detector',
rules = [{
    'detectorId' : 'sample_detector',
    'ruleId' : 'high_fraud_risk',
    'ruleVersion' : '1'
},
{
    'detectorId' : 'sample_detector',
    'ruleId' : 'low_fraud_risk',
    'ruleVersion' : '1'
}
],
modelVersions = [{
    'modelId' : 'sample_fraud_detection_model',
    'modelType': 'ONLINE_FRAUD_INSIGHTS',
    'modelVersionNumber' : '1.00'
}],
ruleExecutionMode = 'FIRST_MATCHED'
)

## Activate the detector

In [ ]:
fraudDetector.update_detector_version_status(
detectorId = 'sample_detector',
detectorVersionId = '1',
status = 'ACTIVE'
)